In [9]:
#specify the number of data you want to load
dataset_size=2000

In [10]:
import pandas as pd
import numpy as np
loc=pd.read_csv('requests.csv')
#working with a subset of data
loc=loc.iloc[:dataset_size,:]
loc.reset_index(inplace=True)
loc.drop('index',axis=1 ,inplace=True)

In [11]:
#task1
from geopy.distance import geodesic
total_distance=0
for i in range(len(loc)):
    pic_loc=(loc.loc[i, 'pick_lat'],loc.loc[1, 'pick_lon'])
    drop_loc=(loc.loc[i, 'drop_lat'],loc.loc[1, 'drop_lon'])
    distance=geodesic(pic_loc,drop_loc).km
    total_distance+=distance
print(total_distance)


4916.139674934178


In [12]:

#task2-small data set
if len (loc)<100:
    pick=[]
    drop=[]
    for i in range(len(loc)):
        pic_loc=(loc.loc[i, 'pick_lat'],loc.loc[1, 'pick_lon'])
        drop_loc=(loc.loc[i, 'drop_lat'],loc.loc[1, 'drop_lon'])
        pick.append(pic_loc)
        drop.append(drop_loc)
    dist=pd.DataFrame({'pick_up':pick,'drop_off':drop})
    
    import time
    from pulp import *
    start_time = time.time()
    passengers=range(len(dist))
    model=LpProblem("share_riding", LpMinimize)
    x=LpVariable.dicts('xij',[(i,j)for i in passengers for j in passengers if i!=j],cat='Binary')
    m=LpVariable.dicts('mij',[(i,j) for i in passengers for j in passengers if i!=j],cat='Binary')
    n=LpVariable.dicts('nij',[(i,j) for i in passengers for j in passengers if i!=j],cat='Binary')
    f=LpVariable.dicts('fi',[i for i in passengers],cat='Binary')

    model+=(lpSum([geodesic((dist.iloc[i,0]),(dist.iloc[j,0])).km *x[(i,j)]for i in passengers for j in passengers if i!=j])
            +lpSum([(geodesic((dist.iloc[j,0]),(dist.iloc[i,1])).km + geodesic((dist.iloc[i,1]),(dist.iloc[j,1])).km) *m[(i,j)]for i in passengers for j in passengers if i!=j ])
            +lpSum([(geodesic((dist.iloc[j,0]),(dist.iloc[j,1])).km+ geodesic((dist.iloc[j,1]),(dist.iloc[i,1])).km) *n[(i,j)]for i in passengers for j in passengers if i!=j ])
            +lpSum([geodesic((dist.iloc[i,0]),(dist.iloc[i,1])).km *f[i] for i in passengers]))



    for i in passengers:
        for j in passengers:
            if i!=j:
                model+= m[(i,j)]+n[(i,j)]==x[(i,j)]
    for i in passengers:
        model+=lpSum([x[(i,j)]  for j in passengers if i!=j])+f[i]+ lpSum([x[(j,i)]  for j in passengers if j!=i])==1

    model.solve()
    shared_ride=0
    for i in passengers:
            for j in passengers:
                if i!=j:
                        if(x[(i,j)].varValue)!=0:
                            shared_ride+=1

    print("running time in soconds", (time.time() - start_time))
    print("total travel distance:" ,value(model.objective))
    print("number of shared rides:", shared_ride)


    
#task2-medium dataset
if 100<=len(loc)<1000:
    pick=[]
    drop=[]
    for i in range(len(loc)):
        pic_loc=(loc.loc[i, 'pick_lat'],loc.loc[1, 'pick_lon'])
        drop_loc=(loc.loc[i, 'drop_lat'],loc.loc[1, 'drop_lon'])
        pick.append(pic_loc)
        drop.append(drop_loc)
    dist=pd.DataFrame({'pick_up':pick,'drop_off':drop})
   
    import time
    from pulp import *
    start_time = time.time()

    passengers=range(len(dist))
    model=LpProblem("share_riding", LpMinimize)
    f=LpVariable.dicts('fi',[i for i in passengers],cat='Binary')
    x={}
    m={}
    n={}
    for i in passengers:
        for j in passengers:
            if i!=j:
                if (geodesic((dist.iloc[i,0]),(dist.iloc[j,0])).km<=0.5 and geodesic((dist.iloc[i,1]),(dist.iloc[j,1])).km<=0.5):
                    x1=LpVariable.dicts('xij',[(i,j)],cat='Binary')
                    m1=LpVariable.dicts('mij',[(i,j)],cat='Binary')
                    n1=LpVariable.dicts('nij',[(i,j)],cat='Binary')
                    if (j,i) not in x.keys():
                        x[(i,j)]=x1[(i,j)]
                        m[(i,j)]=m1[(i,j)]
                        n[(i,j)]=n1[(i,j)]

    model+=(lpSum([geodesic((dist.iloc[i,0]),(dist.iloc[j,0])).km *x[(i,j)]for i in passengers for j in passengers if (i,j)in x.keys()])
            +lpSum([(geodesic((dist.iloc[j,0]),(dist.iloc[i,1])).km + geodesic((dist.iloc[i,1]),(dist.iloc[j,1])).km) *m[(i,j)]for i in passengers for j in passengers if (i,j)in x.keys()])
            +lpSum([(geodesic((dist.iloc[j,0]),(dist.iloc[j,1])).km+ geodesic((dist.iloc[j,1]),(dist.iloc[i,1])).km) *n[(i,j)]for i in passengers for j in passengers if (i,j)in x.keys()])
            +lpSum([geodesic((dist.iloc[i,0]),(dist.iloc[i,1])).km *f[i] for i in passengers]))



    for i in passengers:
        for j in passengers:
            if (i!=j and  (i,j) in x.keys()):
                model+= m[(i,j)]+n[(i,j)]==x[(i,j)]
    for i in passengers:

        model+=lpSum([x[(i,j)]  for j in passengers if ( i!=j and (i,j)  in x.keys()) ])+f[i]+ lpSum([x[(j,i)]  for j in passengers if ( i!=j and (j,i)  in x.keys())])==1  

    model.solve()

    shared_ride=0
    for i in passengers:
        for j in passengers:
            if i!=j:
                if (i,j)in x.keys():
                    if(x[(i,j)].varValue)!=0:
                        shared_ride+=1
    print("running time in soconds", (time.time() - start_time))
    print("total travel distance:" ,value(model.objective))
    print("number of shared rides:", shared_ride)
    
    
#task2- large dataset
if len(loc)>=1000:
    pick=[]
    drop=[]
    for i in range(len(loc)):
        pic_loc=(loc.loc[i, 'pick_lat'],loc.loc[1, 'pick_lon'])
        drop_loc=(loc.loc[i, 'drop_lat'],loc.loc[1, 'drop_lon'])
        pick.append(pic_loc)
        drop.append(drop_loc)
    distance=pd.DataFrame({'pick_up':pick,'drop_off':drop})
    # clustering
    from sklearn.cluster import KMeans
    import numpy as np
    num_cluster=int(len(loc)/10)
    num_pop=len(loc)
    cluster_size=int(num_pop/num_cluster)
    kmn=KMeans(n_clusters=num_cluster)
    x=np.asarray([[loc.loc[i, 'pick_lat'],loc.loc[i, 'pick_lon']] for i in range(len(loc))])
    kmn.fit(x)
    labels=kmn.predict(x)


    clusters={}
    for i in range(num_cluster):
        clusters[i]=[]
        for j in range(len(loc)):
            if labels[j]==i:
                clusters[i].append(j)
    import time
    from pulp import *
    start_time = time.time()
    total_travel=0
    total_shared_ride=0
    total_single_ride=0
    for c in range(num_cluster):
        dist=distance.iloc[clusters[c],:]
        dist.reset_index(inplace=True)
        dist.drop('index',axis=1,inplace=True)
        passengers=range(len(dist))
        model=LpProblem("share_riding", LpMinimize)
        f=LpVariable.dicts('fi',[i for i in passengers],cat='Binary')
        x={}
        m={}
        n={}
        for i in passengers:
            for j in passengers:
                if i!=j:
                    if (geodesic((dist.iloc[i,0]),(dist.iloc[j,0])).km<=0.5 and geodesic((dist.iloc[i,1]),(dist.iloc[j,1])).km<=0.5):
                        x1=LpVariable.dicts('xij',[(i,j)],cat='Binary')
                        m1=LpVariable.dicts('mij',[(i,j)],cat='Binary')
                        n1=LpVariable.dicts('nij',[(i,j)],cat='Binary')
                        if (j,i) not in x.keys():
                            x[(i,j)]=x1[(i,j)]
                            m[(i,j)]=m1[(i,j)]
                            n[(i,j)]=n1[(i,j)]

        model+=(lpSum([geodesic((dist.iloc[i,0]),(dist.iloc[j,0])).km *x[(i,j)]for i in passengers for j in passengers if (i,j)in x.keys()])
                +lpSum([(geodesic((dist.iloc[j,0]),(dist.iloc[i,1])).km + geodesic((dist.iloc[i,1]),(dist.iloc[j,1])).km) *m[(i,j)]for i in passengers for j in passengers if (i,j)in x.keys()])
                +lpSum([(geodesic((dist.iloc[j,0]),(dist.iloc[j,1])).km+ geodesic((dist.iloc[j,1]),(dist.iloc[i,1])).km) *n[(i,j)]for i in passengers for j in passengers if (i,j)in x.keys()])
                +lpSum([geodesic((dist.iloc[i,0]),(dist.iloc[i,1])).km *f[i] for i in passengers]))



        for i in passengers:
            for j in passengers:
                if (i!=j and  (i,j) in x.keys()):
                    model+= m[(i,j)]+n[(i,j)]==x[(i,j)]
        for i in passengers:

            model+=lpSum([x[(i,j)]  for j in passengers if ( i!=j and (i,j)  in x.keys()) ])+f[i]+ lpSum([x[(j,i)]  for j in passengers if ( i!=j and (j,i)  in x.keys())])==1
        #solver = getSolver('GLPK', timeLimit=1)
        model.solve()

        total_travel+=value(model.objective)
        single_ride=0
        shared_ride=0
        for i in passengers:
            if (f[i].varValue)!=0:
                single_ride+=1 
        for i in passengers:
            for j in passengers:
                if i!=j:
                    if (i,j)in x.keys():
                        if(x[(i,j)].varValue)!=0:
                            shared_ride+=1
        total_single_ride+=single_ride
        total_shared_ride+=shared_ride

    print("running time in soconds", (time.time() - start_time))
    print("total travel distance:" ,total_travel)
    print("number of single rides:" ,total_single_ride)
    print("number of shared rides:", total_shared_ride)



    

C:\Users\Marzieh\anaconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


running time in soconds 25.762908935546875
total travel distance: 3561.136301500436
number of single rides: 606
number of shared rides: 697
